# 检测

## 分类
分类问题主要是前景的物体

背景不关心

我们希望前景多,背景少

也就是说分类问题应用比较少

## 检测 Detection

### 不仅分类classification 

需要  n + 1

这个1是背景

### 还要定位location 

框框 叫 bonding box

需要位置(中心点x,y,长,宽)或者(左上xy,右下xy)



## 两阶段检测

比较符合人的思维


# 检测问题常见数据集
> 检测问题是监督学习,  需要数据集

常见分类数据集:
- ImageNet
- Mnist
- CIFAR 10/100

检测数据集:
- Imagenet也有, 质量不好, 但是不常用

- Pascal VOC 
    - 两个版本2007, 2012
    - 2005年公布, 从07年开始一年一次竞赛, 评分标准mAP, 12年竞赛截止
    - pattern analysis statistical modeling and computational learning visual object classes
    - 大概10k张图片, 20个类别, 每个图2-3个obj, 共20k-30kobj

- COCO
    - 微软
    - 200k张图, 80类, 500-600k 大小20g
    - 小目标, 不中心对称, 重叠, 比较难
    - common objects in context

## 传统机器学习如何检测
1. Batch x Channel x Height x Width的图片张量
2. 提取特征, 比如Haris, SIFT, HOG
3. 特征进一步筛选, 聚类, 比如RANSAC
4. 得到一些向量, 输入到SVM, NN, 决策树等机器学习工具
5. 两个任务, 分类, 坐标回归(定位)

每一步都要人工干预, 都很关键, 前面错了后面就白做了

![1](1.png)
结果
- TP: 正确框出物体
- FP: 图中确实有某个物体,框错了
- FN: 本来是背景, 但是标错了. 相当于把物体标成了背景, 但是背景一般是没有框的, 也就是一个物体没画框, 相当于漏检
- TN: 正确框了背景, 背景不用标其实

## CNN做检测
## 思路
应该先定位, 大化小, 多化少, 再分类

关键是定位怎么做呢

- 前景分隔

## 2014第一次尝试 R-CNN
![2](2.png)

把特征向量的提取由传统算法换成cnn

1. 区域建议 Region Proposal
- Selective Search算法(很复杂,很难用)
    - 把图片中纹理, 颜色相近的区域相连接
    - 区块进行区域建议
- 生成大约2000个区域
    - 为了保证需要的都在里面, 不得已,必须这么多
- 每一个都要做resize, 把图片尺寸固定一个大小(因为CNN中FC要求图片参数固定)
    - 弊端: 蛇, 长杆容易改变特征
- 进入二次定位的分类网络, 也就是下一步了
2. 用cnn提取特征向量
    - 这里是AlexNet, **只拿了特征提取层**
    - 每张图片一张一张送入 
    - 后面多加了几层,比如FC, Softmax
    - 先做了pretrain, 然后finetune
    - 用真实数据(图片和bonding box做iou计算得到真实数据)进行finetune, 主要更新后面的层的参数
        > iou是intersection over union, 表征重叠程度, 程度高越认为是前景
        > iou是交并比
        >
        > 图中红框是真实的ground truth
        >
        > 蓝框是区域建议出来的
        >
        > 交集绿色/总和, 也就是: 绿/(红+蓝-绿)
        >
        > ![3](3.png)
        > 
        > 一个经验化的标准, >0.7的正样本, <0.3 认为是负样本
        > 
    - 此时batch=128=32有物体+96没物体的(1:3是经验)
3. 分类(SVM) + 2次定位(线性模型+正则化项)
    - 有softmax还要再设置svm, 这是因为softmax很差, 把softmax丢掉用svm好很多
        - 因为正样本就是原始的正值(ground truth), 而负样本是<iou的, 但是这里负样本的iou比cnn的负样本的iou小, 数量更少, 质量更高(CNN那边不得已而为之,保证数据够多够用, 否则,数据不够还那么复杂网络, 会过拟合)



## 4. NMS算法 Non-Maximum Suppression

> 2014的这个还用了NMS算法
>
> 至今还在用
>
> 这是个精简bonding box个数的算法

原则:

1. 每个框Score越高, 越想要
2. 框与框的iou越大,这两个框越可能是一个物体

也就是选出Score高的框, 然后丢掉和它iou高的其他框

这个阈值越低越严格

Input: 
- score_list: \[score_0, score_1, ..., score_n\]
- bbox_list: \[bbox_0, bbox_1, ..., bbox_n \]
- threshold

Process:

- 找到bbox_list中score_list最大的bbox_m, bbox_list删掉它, 并把它放到new_list中
    - 遍历bbox_list, 每个bbox_i和刚才的bbox_m做iou, 大于threshold, bbox_list就删掉bbox_i(相应的score_list也要删掉对应的)
- 直到bbox_list为空
- 返回new_list


### 传统NMS弊端与改进

![10](10.png)

- 这两个框互斥, 某个score高另一个就可能不会被选中
- NMS只能用CPU
- NMS依赖阈值, 阈值是个很经验的东西, 大了容易漏检, 小了容易误检

### Soft NMS
#### 传统的NMS:
>  大于threshold, bbox_list就删掉(相应的score_list也要删掉对应的)

这里score_list的删除和保留操作, 相当于:
$$
\begin{align*}
 &S_{i}=
 \left\{\begin{matrix}
 S_{i}&if\; iou(bbox_{m},bbox_{i})<threshold \\ 
0 & if\; iou(bbox_{m},bbox_{i})\geqslant threshold
\end{matrix}\right. 
\\ 
 &= 
 S_{i}\left\{\begin{matrix}
 1 &if\; iou(bbox_{m},bbox_{i})<threshold \\ 
0 & if\; iou(bbox_{m},bbox_{i})\geqslant threshold
\end{matrix}\right.
\end{align*}

$$

也就是传统是"硬"的, 粗暴的0和1

#### Soft NMS不做删除:
$$
S_{i}=S_{i} \cdot \left\{\begin{matrix}
 1&if\; iou(bbox_{m},bbox_{i})<threshold \\ 
 1-iou & if\; iou(bbox_{m},bbox_{i})\geqslant threshold
\end{matrix}\right.
$$

可以看出1-iou是个关于iou的递减函数

所以我们甚至可以写成一个式子, 并且摆脱阈值: 其中sigma 很小

$$ S_{i}=S_{i} \cdot exp(- \frac{iou^2}{\sigma})$$
Input: 
- score_list: \[score_0, score_1, ..., score_n\]
- bbox_list: \[bbox_0, bbox_1, ..., bbox_n \]
- threshold

Process:

- 找到bbox_list中score_list最大的bbox_m, bbox_list删掉它, 并把它放到new_list中
- 遍历bbox_list, 每个bbox_i和刚才的bbox_m做iou, score_list对应的就要变小, 变小的系数和iou有关

#### 注意: 
- 之后可能需要筛选score的阈值
- 训练时不用soft, 测试时用更好, 说明易用性强

### 5. Bonding Box Regression
对偏移量做回归

归一化

参看[Faster RCNN](#Faster)

##  2014 - RCNN 缺点&问题:


- 2000次输入CNN卷积, 太慢(大概一分钟一张),  重合的地方没必要重复卷积
- 用SVM不好
- 计算复杂, 结构臃肿, 一会儿用cnn, 一会儿丢弃softmax, 一会儿用SVM

## 改进 - 2015 - Fast RCNN 

> 每秒1-2张

![11](11.png)

1.  原本selective search生成2000个区域, 每个去卷积

    现在原图还是selective search生成2000个区域, 投影到特征图上(RoI projection. region of interest), 特征图做一次卷积

    本质上也就是对特征图分割了, 而不是原图

    投影时, 要注意卷积网络的pooling, 将尺寸变小了, 所以投影也要把尺寸相应变小, 比如vgg16, 就需要/16
    
    
2.  投影到特征图上的小区域还是不一样小的, 需要变成一样小, 
    
    resize不行, 长条会失去形状
    
    用RoI pooling layer(RoI pooling仍然可以改进)

3.  SVM -> Softmax

4.  训练的时候, IoU>0.5的是正样本, IoU在0.1-0.5的是负, 0-0.1的用于验证. 

    验证错误的例子重新训练. 这叫Hard Example Mining

5.  两个分支输出, 因此有多个loss. 

    位置,坐标回归Loss用L1,以避免L2大的更大小的更小; 
    
    分类的Loss用交叉熵

    注意, 背景没有位置的loss, 所以可以用01系数来算

### ROI Pooling 讨论

投影之后需要同样缩小尺寸, 如前文的/16

可能会取整

然后变成尺寸一样大的, 这里需要用网格划分

每个网格做max pooling方面

![12](12.png)

但是, 比如5x7的变成2x2的, 也会取整

取整会丢失信息, 尤其是小的区域

### 如何改进
1. 投射方面

2. RoI pooling方面


### RoI align
![13](13.png)

投射的时候/16, 得到小数点, 没关系, 保留浮点数

网格划分, N=4(经验), 每个小网格(bin)内取N块

bin的max值就是N个块的max的max

取每个小块的正中间的点,代表这个块

这个点的坐标一般是浮点数了

这个点显然附近有4个坐标为整数的像素点(确切来说叫特征点)

这个点的值,用4个点的双线性插值

$$ $$
### 缺点

N超参数,要提前指定

并不是所有点贡献进来了, 并不一定是每个块的中心点附近有最值

### precise roi (2018-iou-net)
也是两次都保留浮点

对于bin, 其实存在虚点, 和原来的特征点有位置偏差

用双线性插值, 根据周围每一个虚点4个实点, 计算每一个的虚点

看起来计算量大,但是用到了很多重复的点, 算法可以优化的

算完之后求average pooling

<span id='Faster'></span>
## 进一步改进 Faster RCNN 2015

[论文地址](https://arxiv.org/pdf/1506.01497.pdf)

> 里程碑, 很重要
>
> 每秒10帧
>
> 起初为了改变selective search 的效率低下问题
>
> 思考: 能不能用CNN代替那些复杂的数学计算呢?

### 结构概述
![15](15.png)

1. backbone 用CNN将image变成feature map
2. RPN, 将feature map变成region proposal或者RoI
3. RoI pooling 然后用 Fast RCNN得出回归和分类的loss, 迭代训练


### backbone + rpn 的测试阶段:
![30](30.png)

1.  用backbone得到feature map, 比如是WxHxC

2.  先用cx3x3xn的卷积, 得到WxHxn. 这一步是语义转换, 增加rpn的拟合能力, n一般512

3.  然后分支

    18= 9x2 分类信息

    36= 9x4 坐标回归信息

    这个2和4固定的

    9是9个anchor值

    anchor 对应原图某个位置,  这个位置可能有2个分类概率和4个坐标回归

3.1 nx1x1x18的卷积, 得到WxHx18

    18的先reshape成 2xWxHx9 做softmax, 再reshape回去

    这个reshape是原文中caffe特有, torch可以不用

    2是因为2分类(是物体还是不是物体)

3.2 另一个分支 nx1x1x36的卷积

4.  合并 , 得到region proposals, 这里我们只关心这个是前景还是背景

> 两阶段的含义: 1找出物体,2找出物体是什么类别



#### anchor


现在每个特征点对应原图9个框

3种w和h比例:
- 1:1
- 2:1
- 1:2

3种特征点大小和原图比例:
- 16
- 256
- 512

组合一下就是9个

例如 , 如果特征图是38x50 的

那么就有9x38x50=17100个anchor

具体情况下, 根据训练集的label, 可以更改那些比例

比如蛇, 10:1, 1:10 , 1:20

###  RPN训练阶段
![40](40.png)

用特征图创建原图的anchor, 叫做create anchor, 比如17100个

拿出ground truth(这个里面只有框)

anchor和gt做iou和nms, 得到9x2和9x4, 相当于特征点的anchor_label

和rpn中得到的9x2, 9x4分别做cls_loss, loca_loss

这样就可以训练rpn

$$Loss = cls\_loss + \lambda loca\;\_loss$$

\lambda 常取0.1

#### loca_loss 之 smooth L1 loss
loca_loss使用, 当然只有类别是物体才算这个loss

$$
f(x)=\left\{\begin{matrix}
\frac{(\sigma x)^2}{2} &  if \; x<\frac{1}{\sigma^2}\\
|x|-\frac{1}{2 \sigma^{2}} & otherwise
\end{matrix}\right.
$$

#### Bonding Box Regression

bbox两种数据结构:
1. 中心点(x, y)和w,h
2. x1, y1, x2, y2

在L1 中, 要预测偏移量, 也要对大小物体要归一化



![51](51.png)
![50](50.png)

create anchor 生成的是a

走模型预测出来的是p

我们本来想训练这个网络如何从a到t

但其实是p->t

迁移预测 p-a 去接近t-a

以第一种为例: 

$$x_{p-a}=\frac{x-x_a}{w_a}$$

$$y_{p-a}=\frac{y-y_a}{h_a}$$

$$w_{p-a}=\log \frac{w}{w_a}$$

$$h_{p-a}=\log \frac{h}{h_a}$$

$$x_{t-a}=\frac{x^*-x_a}{w_a}$$

$$y_{t-a}=\frac{y^*-y_a}{h_a}$$

$$w_{t-a}=\log \frac{w^*}{w_a}$$

$$h_{t-a}=\log \frac{h^*}{h_a}$$

最终, 对于某一个物体:

$$loca\_loss = \frac{1}{4}\sum_{i\in\{x,y,w,h\}}^{4} f(i_{p-a}-i_{t-a})$$

#### cls_loss

> 9x4解决了, 那么9x2怎么解决呢?

很多anchor比如17100个, 过边界筛除, 太小筛除, 1w左右

NMS筛除

然后IoU筛除, >0.7正样本, <0.3负样本标签

得到128个框,含有物体: 不含物体=1:3

### Fast Rcnn阶段

rpn中得到9x2和9x4后, 要和feature map结合

做roi pooling

输入到fast rcnn中

(其实这里相当于拿到了region proposal, 并且是比selective search牛逼的region proposal)


让这些小的特征图再进行回归和分类(20)

![60](60.png)

#### 存在问题:

训练rpn的时候 backbone变了

fast rcnn 也包含这个backbone, 同时用到了rpn的输出

所以, 用rpn的数据来训练fast rcnn, backbone又变了

#### 联合训练alternative training:

1. 先训练rpn, 包括backbone和后面那些512, 18, 36等特有部分
2. 训练rcnn ,此时会改变backbone
3. 使用2中的backbone, 重新训练rpn, 此时保持backbone不变, 只改变特有部门
4. 重新训练rcnn,固定整个rpn都不变



### faster rcnn GitHub复现

[最简单的用torch复现的,作者: chenyuntc](github.com/chenyuntc/simple-faster-rcnn-pytorch)

[腾讯open-mmlab, 图像检测工具包](github.com/open-mmlab/)

[detectron2, 挺新的](github.com/facebookresearch/detectron2/)

# TODO
怎么由gt和create anchor得到9x4和9x2的?

这是针对每一个特征点的吗?

predict中如何产生9x4和9x2的?

128又是什么?